In [104]:
#import statements
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [105]:
# standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
# html_for_page = requests.get(standings_url)
# soup = BeautifulSoup(html_for_page.text)
# parentTable = soup.select('table.stats_table')[0]
# parentTable

In [106]:
# teams_list = parentTable.find_all('a')
# teams_list
# teams_url = [l.get('href') for l in teams_list]
# teams_url = [l for l in teams_url if '/squads/' in l]
# urls = [f"https://fbref.com{l}" for l in teams_url]
# urls

In [107]:
# data = requests.get(urls[0])
# scores_table = pd.read_html(data.text, match='Scores & Fixtures')[0]
# soup = BeautifulSoup(data.text)
# team_data = soup.find_all('a')
# fresh_data= [team.get('href')for team in team_data]
# fresh_data = [l for l in fresh_data if l and 'all_comps/shooting/' in l]
# fresh_data = [f"https://fbref.com{l}" for l in fresh_data]
# data = requests.get(fresh_data[0])
# shooting_table = pd.read_html(data.text, match='Shooting')[0]
# shooting_table.columns = shooting_table.columns.droplevel()
# shooting_table

In [108]:
# team_data = scores_table.merge(shooting_table[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [109]:
#now we deal with all the tables across multiple seasons
years = list(range(2022, 2020, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]

        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(5)

In [111]:
standings_table = soup.select('table.stats_table')
standings_table[0]

<table class="stats_table sortable min_width" data-cols-to-freeze=",1" id="stats_standard_9"> <caption>Standard Stats <span style="color: #666; font-size:smaller">2022-2023 Southampton: Premier League</span> Table</caption> <colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup> <thead> <tr class="over_header"> <th aria-label="" class="over_header center" colspan="4" data-stat=""></th><th></th> <th aria-label="" class="over_header center group_start" colspan="3" data-stat="header_playing">Playing Time</th> <th aria-label="" class="over_header center group_start" colspan="8" data-stat="header_performance">Performance</th> <th aria-label="" class="over_header center group_start" colspan="4" data-stat="header_expected">Expected</th> <th aria-label="" class="over_header center group_start" colspan="3" data-stat="header_pro

In [112]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df.to_csv("matches.csv")